# **FOREWORD**

This is a starter kernel for the [Playground Season-5 Episode 8](https://www.kaggle.com/competitions/playground-series-s5e8) binary competition with auc metric. This needs to be maximized. 

We start off with simple EDA and ML models and then improve the results as time passes! Let's get started! <br>
In this version, I use a simple offline ML models with 5-folds and make a submission. Let's see how this goes!

#### **NOTE**
I have simplified my ModelTrainer class for the concurrent playground episode. Please peruse it and provide me relevant feedback.

# **IMPORTS**

In [1]:
!uv pip install -q --system -r /kaggle/input/playgrounds5e8-public-imports-v1/req_kaggle.txt

season  = 5
episode = 8

exec( open(f"/kaggle/input/playgrounds{season}e{episode}-public-imports-v1/myimports.py", "r").read() )
exec( open(f"/kaggle/input/playgrounds{season}e{episode}-public-imports-v1/myutils.py", "r").read() )
exec( open(f"/kaggle/input/playgrounds{season}e{episode}-public-imports-v1/training.py", "r").read() )
exec( open(f"/kaggle/input/playgrounds{season}e{episode}-public-imports-v1/mypp.py", "r").read() )

print()

---> Imports- part 1 done
---> Sklearn = 1.7.1 | Pandas = 2.2.3 | Polars = 1.31.0
---> Commencing imports-part2
---> XGBoost = 3.0.2 | LightGBM = 4.6.0
---> Imports- part 2 done
---> Seeding everything

---> Imports done




# **CONFIGURATION**

In [2]:
%%time

class CFG:
    """
    Configuration class for parameters and CV strategy for tuning and training
    Some parameters may be unused here as this is a general configuration class
    """

    # Data preparation:-
    version_nb         = 1
    model_id           = "V1_2"
    model_label        = "ML"
    test_req           = False
    test_iter          = 50
    gpu_switch         = "ON" if torch.cuda.is_available() else "OFF"
    state              = 42
    target             = f"y"
    grouper            = f""
    
    tgt_mapper         = {}
    
    ip_path            = f"/kaggle/input/playground-series-s5e8"
    op_path            = f"/kaggle/working"
    orig_path          = f"/kaggle/input/bank-marketing-dataset-full/bank-full.csv"
    data_path          = f""
    dtl_preproc_req    = True
    ftre_plots_req     = False
    ftre_imp_req       = False
    nb_orig            = 0
    orig_all_folds     = False

    # Model Training:-
    pstprcs_oof        = False
    pstprcs_train      = False
    pstprcs_test       = False
    ML                 = True
    test_preds_req     = True
    n_splits           = 10
    n_repeats          = 1
    nbrnd_erly_stp     = 250
    mdlcv_mthd         = 'SKF'
    metric_obj         = 'maximize'

    # Global variables for plotting:-
    grid_specs = {'visible'  : True,
                  'which'    : 'both',
                  'linestyle': '--',
                  'color'    : 'lightgrey',
                  'linewidth': 0.75
                 }

    title_specs = {'fontsize'   : 9,
                   'fontweight' : 'bold',
                   'color'      : '#992600',
                  }

cv_selector = \
{
 "RKF"   : RepeatedKFold(n_splits = CFG.n_splits, n_repeats= CFG.n_repeats, random_state= CFG.state),
 "RSKF"  : RepeatedStratifiedKFold(n_splits  = CFG.n_splits, n_repeats= CFG.n_repeats, random_state= CFG.state),
 "SKF"   : StratifiedKFold(n_splits = CFG.n_splits, shuffle = True, random_state= CFG.state),
 "KF"    : KFold(n_splits = CFG.n_splits, shuffle = True, random_state= CFG.state),
 "GKF"   : GroupKFold(n_splits = CFG.n_splits)
}

collect()

CPU times: user 194 ms, sys: 48.7 ms, total: 243 ms
Wall time: 285 ms


0

## **KEY CONFIG PARAMETERS**

|Configuration parameter| Explanation| Data type| Sample values |  
| ---------------------- | ------------------------------- | --------------------- | --------------- |
| version_nb    | Version Number | int | 1 | 
| model_id      | Model ID    | string | V1_1 | 
| model_label   | Model Label | string | ML | 
| test_req      | Test Required| bool | True / False | 
| test_iter     | Test case iterations for models | int | 50 |
| gpu_switch      | Do we need GPU support | bool | True / False |
| state           | Random state | int | 42 |
| target          | Target column | str |  |
| grouper         | CV grouper column | str |  |
| ip_path, op_path | Data paths  | str | |
| pstprcs_* | Do we need post-processing  | bool |True / False |
| ML| Do we need machine learning models  | bool |True / False |
| test_preds_req| Do we need test set predictions (training in inference kernel)  | bool |True / False |
| n_splits/ n_repeats | N-splits and repeats for CV scheme | int | 3/5/10|
| nbrnd_erly_stp | Early stopping rounds | int | 40|
| mdlcv_mthd | Model CV method | str | RSKF|
| ensemble_req | Do we need ensemble | bool | True / False |
| metric_obj   | Metric direction | str | minimize/ maximize |

# **PREPROCESSING**

In [3]:
%%time 

pp = Preprocessor()
pp.DoPreprocessing();

Data shapes - train-test-original | (750000, 17) (250000, 16) (45211, 17)

Train set head


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
id,,,,,,,,,,,,,,,,,
0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1



Test set head


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
id,,,,,,,,,,,,,,,,
750000,32,blue-collar,married,secondary,no,1397,yes,no,unknown,21,may,224,1,-1,0,unknown
750001,44,management,married,tertiary,no,23,yes,no,cellular,3,apr,586,2,-1,0,unknown
750002,36,self-employed,married,primary,no,46,yes,yes,cellular,13,may,111,2,-1,0,unknown
750003,58,blue-collar,married,secondary,no,-1380,yes,yes,unknown,29,may,125,1,-1,0,unknown
750004,28,technician,single,secondary,no,1950,yes,no,cellular,22,jul,181,1,-1,0,unknown



Original set head


...


Original information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  int64 
 17  Source     45211 non-null  object
dtypes: int64(8), object(10)
memory usage: 6.2+ MB


None


Unique and null values



,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Source
Train_Nunq,78,12,3,4,2,"8,217",2,2,3,31,12,"1,760",52,596,50,4,1
Test_Nunq,76,12,3,4,2,"7,073",2,2,3,31,12,"1,620",49,518,47,4,1
Original_Nunq,77,12,3,4,2,"7,168",2,2,3,31,12,"1,573",48,559,41,4,1
Train_Nulls,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Test_Nulls,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Original_Nulls,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0



We are using the competition training data only
CPU times: user 4.47 s, sys: 787 ms, total: 5.26 s
Wall time: 6.42 s


# **FEATURE CREATION**

We do the below here - 
- Convert all columns to integer type using Ordinal Encoder
- Reduce memory for the datasets


In [4]:
%%time 

Xtrain    = pp.train.drop(CFG.target, axis=1)
Xtest     = pp.test.copy()
ytrain    = pp.train[CFG.target]
PrintColor(f"---> Shapes = {Xtrain.shape} {Xtest.shape}")

num_cols = Xtest.select_dtypes(np.number).columns.tolist()
cat_cols = deepcopy(pp.cat_cols)

Xtrain[pp.cat_cols] = Xtrain[pp.cat_cols].astype("string")
Xtest[pp.cat_cols]  = Xtest[pp.cat_cols].astype("string")

print(f"---> Shapes = {Xtrain.shape} {Xtest.shape}")

Xtrain = reduce_mem_usage(Xtrain, "Train")
Xtest  = reduce_mem_usage(Xtest, "Test")

_ = utils.CleanMemory()

---> Shapes = (750000, 17) (250000, 17)
---> Shapes = (750000, 17) (250000, 17)
---> Reducing memory usage for: Train


  0%|          | 0/17 [00:00<?, ?it/s]

---> Memory usage before: 97.27 MB
---> Memory usage after: 66.52 MB
---> Decreased memory usage by 31.6176 percent 

---> Reducing memory usage for: Test


  0%|          | 0/17 [00:00<?, ?it/s]

---> Memory usage before: 34.33 MB
---> Memory usage after: 24.08 MB
---> Decreased memory usage by 29.8611 percent 

CPU times: user 2.91 s, sys: 158 ms, total: 3.07 s
Wall time: 3.05 s


# **CV-SCHEME**

In [5]:
%%time 

# Initializing the cv scheme:-
cv = cv_selector[CFG.mdlcv_mthd]

if CFG.nb_orig > 0:
    all_df = []
    
    for mysource in ["Competition", "Original"]:
        df = pd.concat([Xtrain.loc[Xtrain.Source == mysource], ytrain], axis=1, join = "inner")
        df.index = range(len(df))
        for fold_nb, (_, dev_idx) in enumerate(cv.split(df, df[CFG.target])):
            df.loc[dev_idx, "fold_nb"] = fold_nb
            
        all_df.append(df)      
    ygrp = pd.concat(all_df, axis=0, ignore_index = True)["fold_nb"].astype(np.uint8)
                      
else:
    df = Xtrain.loc[Xtrain.Source == "Competition"]
    df.index = range(len(df))
    
    for fold_nb, (_, dev_idx) in enumerate(cv.split(df, ytrain.iloc[df.index])):
        df.loc[dev_idx, "fold_nb"] = fold_nb 
    ygrp = df["fold_nb"].astype(np.uint8)

PrintColor(
    f"\n---> Shapes = {Xtrain.shape} {Xtest.shape} {ytrain.shape} {ygrp.shape}"
)


---> Shapes = (750000, 17) (250000, 17) (750000,) (750000,)
CPU times: user 435 ms, sys: 66 ms, total: 501 ms
Wall time: 505 ms


# **MODEL TRAINING**

In [6]:
%%time 

Mdl_Master = \
{     
 f'XGB1C'  : [
              XGBC(**{ "objective"            : "binary:logistic",
                       "eval_metric"          : "logloss",
                       'device'               : "cuda:0" if CFG.gpu_switch == "ON" else "cpu",
                       'learning_rate'        : 0.02,
                       'n_estimators'         : 10000 if CFG.test_req == False else CFG.test_iter,
                       'max_depth'            : 7,
                       'colsample_bytree'     : 0.70,
                       'verbosity'            : 0,
                       'random_state'         : CFG.state,
                       'early_stopping_rounds': CFG.nbrnd_erly_stp,
                       'enable_categorical'   : True,
                      } 
                   ),
              {"verbose" : 0}
             ],

 f"LGBM1C" : [
                 LGBMC(
                         max_depth           = 4,     
                         n_estimators        = 10_000 if CFG.test_req == False else CFG.test_iter,
                         learning_rate       = 0.075,
                         reg_alpha           = 0.85,
                         reg_lambda          = 3.50,
                         colsample_bytree    = 0.50,
                         subsample           = 0.80,     
                         random_state        = CFG.state,
                         verbosity           = -1,
                 ),
                 {"callbacks" : [log_evaluation(0), early_stopping(CFG.nbrnd_erly_stp, verbose = False)],
                  "eval_metric" : "auc",
                 } 
             ]
}

# Initializing model outputs
OOF_Preds    = {}
Mdl_Preds    = {}

CPU times: user 85 µs, sys: 13 µs, total: 98 µs
Wall time: 101 µs


In [7]:
%%time 

drop_cols = ["Source", "id", "Id", "Label", CFG.target, "fold_nb"]

for method, (mymodel, fit_params) in tqdm(Mdl_Master.items()) :
    md = ModelTrainer(
        drop_cols    = drop_cols, 
        problem_type = "binary",
        len_train    = Xtrain.loc[Xtrain.Source == "Competition"].shape[0]
    )

    _, oof_preds, mdl_preds = \
    md.fit_predict(
        Xtrain.copy(), 
        ytrain,
        Xtest.copy(), 
        ygrp,
        extra    = None,
        method   = method,
        mymodel  = mymodel,
        cat_cols = cat_cols,
        **fit_params,
    )   

    OOF_Preds[method] = oof_preds.flatten()
    Mdl_Preds[method] = mdl_preds.flatten()
    collect();

    score = utils.ScoreMetric(ytrain.values[0 : len(oof_preds)], oof_preds)
    PrintColor(
        f"---> Overall score = {score :,.8f} | Competition metric",
        color = Fore.CYAN
    )

_ = utils.CleanMemory()
print()

  0%|          | 0/2 [00:00<?, ?it/s]


 XGB1C offline model training



XGB1C: 0it [00:00, ?it/s]

---> OOF score = 0.96984655 | Fold 1
---> OOF score = 0.96852580 | Fold 2
---> OOF score = 0.96785105 | Fold 3
---> OOF score = 0.96836375 | Fold 4
---> OOF score = 0.96743776 | Fold 5
---> OOF score = 0.96853740 | Fold 6
---> OOF score = 0.96938423 | Fold 7
---> OOF score = 0.96877210 | Fold 8
---> OOF score = 0.96870145 | Fold 9
---> OOF score = 0.96823642 | Fold 10

---> Overall score = 0.96856565 +- 0.00065621
---> Overall score = 0.96856529 | Competition metric

 LGBM1C offline model training



LGBM1C: 0it [00:00, ?it/s]

---> OOF score = 0.97037624 | Fold 1
---> OOF score = 0.96906701 | Fold 2
---> OOF score = 0.96823765 | Fold 3
---> OOF score = 0.96873768 | Fold 4
---> OOF score = 0.96813658 | Fold 5
---> OOF score = 0.96866111 | Fold 6
---> OOF score = 0.96975827 | Fold 7
---> OOF score = 0.96925667 | Fold 8
---> OOF score = 0.96939121 | Fold 9
---> OOF score = 0.96872911 | Fold 10

---> Overall score = 0.96903515 +- 0.00065298
---> Overall score = 0.96903337 | Competition metric

CPU times: user 1h 26min 7s, sys: 18.7 s, total: 1h 26min 25s
Wall time: 46min 13s


# **CLOSURE**

In [8]:
%%time 

pp.sub_fl[CFG.target] = pd.DataFrame(Mdl_Preds).mean(axis=1).values.flatten()

pp.sub_fl.to_csv("submission.csv", index = None)
print()
!ls submission.csv
print()
!head submission.csv


submission.csv

id,y
750000,0.0014438981
750001,0.06991498
750002,9.7054384e-05
750003,3.7585873e-05
750004,0.017939303
750005,0.0014727113
750006,0.29366007
750007,0.34055412
750008,0.0038179029
CPU times: user 410 ms, sys: 60 ms, total: 470 ms
Wall time: 748 ms
